<a href="https://colab.research.google.com/github/yukinaga/twitter_bot/blob/master/section_6/01_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# データの前処理
対話文のデータセットに前処理を行い、保存します。

## ライブラリのインストール
分かち書きのためにjanomeを、テキストデータの前処理のためにtorchtextをインストールします。

In [ ]:
!pip install janome==0.4.1
!pip install torchvision==0.7.0
!pip install torchtext==0.7.0
!pip install torch==1.6.0

## Google ドライブとの連携  
以下のコードを実行し、認証コードを使用してGoogle ドライブをマウントします。

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

## 対話文の取得
雑談対話コーパス「projectnextnlp-chat-dialogue-corpus.zip」をダウンロードします。  
  
> Copyright (c) 2015 Project Next NLP 対話タスク 参加者一同  
> https://sites.google.com/site/dialoguebreakdowndetection/chat-dialogue-corpus/LICENSE.txt  
> Released under the MIT license

解凍したフォルダをGoogle ドライブにアップします。  
フォルダからjsonファイルを読み込み、対話文として成り立っている文章を取り出してリストに格納します。  



In [ ]:
import glob  # ファイルの取得に使用
import json  # jsonファイルの読み込みに使用
import re

path = "/content/drive/My Drive/live_ai_data/projectnextnlp-chat-dialogue-corpus/json"  # フォルダの場所を指定

files = glob.glob(path + "/*/*.json")  # ファイルの一覧
dialogues = []  # 複数の対話文を格納するリスト
file_count= 0  # ファイル数のカウント
for file in files:
    with open(file, "r") as f:
        json_dic = json.load(f)
        dialogue = []  # 単一の対話
        for turn in json_dic["turns"]:
            annotations = turn["annotations"]  # 注釈
            speaker = turn["speaker"]  # 発言者
            utterance = turn["utterance"]  # 発言

            # 空の文章や、特殊文字や数字が含まれる文章は除く
            if (utterance=="") or ("\\u" in utterance) or (re.search("\d", utterance)!=None):
                dialogue.clear()  # 対話をリセット
                continue

            utterance = utterance.replace(".", "。").replace(",", "、")  # 全角
            utterance = utterance.replace("．", "。").replace("，", "、")  # 半角
            utterance = utterance.split("。")[0]

            if speaker=="U":  # 発言者が人間であれば
                dialogue.append(utterance) 
            else:  # 発言者がシステムであれば
                is_wrong = False
                for annotation in annotations:
                    breakdown = annotation["breakdown"]  # 分類
                    if breakdown=="X":  # 1つでも不適切評価があれば
                        is_wrong = True
                        break
                if is_wrong:
                    dialogue.clear()  # 対話をリセット
                else:
                    dialogue.append(utterance)  # 不適切評価が無ければ対話に追加
            
            if len(dialogue) >= 2:  # 単一の会話が成立すれば
                dialogues.append(dialogue.copy())
                dialogue.pop(0)  # 最初の要素を削除

    file_count += 1
    if file_count%100 == 0:
        print("files:", file_count, "dialogues", len(dialogues))

print("files:", file_count, "dialogues", len(dialogues))

## データ拡張の準備
データ拡張の準備として、正規表現の設定および分かち書きを行います。

In [ ]:
import re
from janome.tokenizer import Tokenizer

re_kanji = re.compile(r"^[\u4E00-\u9FD0]+$")  # 漢字の検出用
re_katakana = re.compile(r"[\u30A1-\u30F4]+")  # カタカナの検出用
j_tk = Tokenizer()

def wakati(text):
    return [tok for tok in j_tk.tokenize(text, wakati=True)] 

wakati_inp = []  # 単語に分割された入力文
wakati_rep = []  # 単語に分割された応答文
for dialogue in dialogues:
    wakati_inp.append(wakati(dialogue[0])[:10])
    wakati_rep.append(wakati(dialogue[1])[:10])

## データ拡張
対話データの数を水増しします。  
ある入力文を、それに対応する応答文以外の複数の応答文と組み合わせます。  
組み合わせる応答文は、入力文に含まれる漢字やカタカナの単語を含むものを選択します。  

In [ ]:
dialogues_plus = []
for i, w_inp in enumerate(wakati_inp):  # 全ての入力文でループ
    inp_count = 0  # ある入力から生成された対話文をカウント
    for j, w_rep in enumerate(wakati_rep):  # 全ての応答文でループ
        if i==j:
            dialogues_plus.append(["".join(w_inp), "".join(w_rep)])
            continue
        similarity = 0  # 類似度
        for w in w_inp:  # 入力文と同じ単語があり、それが漢字かカタカナであれば類似度を上げる
            if (w in w_rep) and (re_kanji.fullmatch(w) or re_katakana.fullmatch(w)):
                similarity += 1
        if similarity >= 1:
            dialogue_plus = ["".join(w_inp), "".join(w_rep)]
            if dialogue_plus not in dialogues_plus:
                dialogues_plus.append(dialogue_plus)
                inp_count += 1
                if inp_count >= 12:  # ある入力から生成する対話文の上限
                    break

    if i%1000 == 0:
        print("i:", i, "dialogues_pus:", len(dialogues_plus))

print("i:", i, "dialogues_pus:", len(dialogues_plus))

拡張された対話データを、新たな対話データとします。

In [ ]:
dialogues = dialogues_plus

## 対話データの保存
対話データをcsvファイルとしてGoogle Driveに保存します。

In [ ]:
import csv
from sklearn.model_selection import train_test_split

dialogues_train, dialogues_test =  train_test_split(dialogues, shuffle=True, test_size=0.05)  # 5%がテストデータ
path = "/content/drive/My Drive/live_ai_data/"  # 保存場所

with open(path+"dialogues_train.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(dialogues_train)

with open(path+"dialogues_test.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(dialogues_test)

## 対話文の取得
Googleドライブから、対話文のデータを取り出してデータセットに格納します。  



In [ ]:
import torch
import torchtext
from janome.tokenizer import Tokenizer

path = "/content/drive/My Drive/live_ai_data/"  # 保存場所を指定

j_tk = Tokenizer()
def tokenizer(text): 
    return [tok for tok in j_tk.tokenize(text, wakati=True)]  # 内包表記
 
# データセットの列を定義
input_field = torchtext.data.Field(  # 入力文
    sequential=True,  # データ長さが可変かどうか
    tokenize=tokenizer,  # 前処理や単語分割などのための関数
    batch_first=True,  # バッチの次元を先頭に
    lower=True  # アルファベットを小文字に変換
    )

reply_field = torchtext.data.Field(  # 応答文
    sequential=True,  # データ長さが可変かどうか
    tokenize=tokenizer,  # 前処理や単語分割などのための関数
    init_token = "<sos>",  # 文章開始のトークン
    eos_token = "<eos>",  # 文章終了のトークン
    batch_first=True,  # バッチの次元を先頭に
    lower=True  # アルファベットを小文字に変換
    )
 
# csvファイルからデータセットを作成
train_data, test_data = torchtext.data.TabularDataset.splits(
    path=path,
    train="dialogues_train.csv",
    validation="dialogues_test.csv",
    format="csv",
    fields=[("inp_text", input_field), ("rep_text", reply_field)]  # 列の設定
    )

## 単語とインデックスの対応
単語にインデックスを割り振り、辞書として格納します。

In [ ]:
input_field.build_vocab(
    train_data,
    min_freq=3,
    )
reply_field.build_vocab(
    train_data,
    min_freq=3,
    )

In [ ]:
print(input_field.vocab.freqs)  # 各単語の出現頻度
print(len(input_field.vocab.stoi))
print(len(input_field.vocab.itos))
print(len(reply_field.vocab.stoi))
print(len(reply_field.vocab.itos))

## データセットの保存
データセットの`examples`とFieldをそれぞれ保存します。

In [ ]:
import dill

torch.save(train_data.examples, path+"train_examples.pkl", pickle_module=dill)
torch.save(test_data.examples, path+"test_examples.pkl", pickle_module=dill)

torch.save(input_field, path+"input_field.pkl", pickle_module=dill)
torch.save(reply_field, path+"reply_field.pkl", pickle_module=dill)